In [71]:
import pandas as pd
import numpy as np
import aiohttp
import asyncio

MAX_TRACES_PER_USER = 5
DESIRED_TRACES = 1000
OUTPUT_FOLDER = "output"

In [2]:
data = pd.read_csv(
    "web_routineness_data/browsing.csv",
    dtype={"id": int, "web_visits_id": int},
    parse_dates=["used_at"],
).drop(columns=["id"])
data.head()

,web_visits_id,panelist_id,url,used_at,active_seconds,domain,subdomain
0,111761051,1421,0,2018-10-05 22:02:38,4,ebesucher.de,NaN
1,111761051,1421,0,2018-10-05 21:41:22,10,ebesucher.de,NaN
2,111761051,1421,0,2018-10-05 21:45:36,5,ebesucher.de,NaN
3,111761051,1421,0,2018-10-05 21:42:22,181,ebesucher.de,NaN
4,111761055,1421,1,2018-10-05 21:47:57,2,klamm.de,NaN


In [3]:
# Sanity check: only keep panelists with a good number of web page visits
REQUIRED_DATAPOINTS = 200

panelists_to_filter = data.groupby("panelist_id").filter(
    lambda x: len(x) < REQUIRED_DATAPOINTS
)["panelist_id"].unique()

print(f"Filtering out {len(panelists_to_filter)} panelists with less than {REQUIRED_DATAPOINTS} datapoints")

data_filtered = data[~data["panelist_id"].isin(panelists_to_filter)].copy()
print("Rows:", len(data_filtered))
data_filtered.sort_values("used_at").iloc[np.r_[0:5, -5:0]]

Filtering out 319 panelists with less than 200 datapoints
Rows: 9127431


,web_visits_id,panelist_id,url,used_at,active_seconds,domain,subdomain
1369019,114016772,1849,498963,2018-10-01 00:00:00,8,nicequest.com,NaN
1315961,113973380,750,481401,2018-10-01 00:00:00,2,vasilisaroom.site,NaN
1232915,113894036,957,454324,2018-10-01 00:00:00,6,google.com,NaN
1520755,114158144,1982,552480,2018-10-01 00:00:00,2,facebook.com,NaN
1346500,113996676,1170,491134,2018-10-01 00:00:01,5,ebay-kleinanzeigen.de,NaN
8884856,124428860,1627,2918269,2018-10-31 23:59:58,16,ebay.de,NaN
8944905,124539076,1067,241123,2018-10-31 23:59:58,10,gfk.de,ecpo.
9091905,124804680,1226,813332,2018-10-31 23:59:58,452,mypersonalroutine.com,de.
8938760,124531492,177,2934742,2018-10-31 23:59:59,6,crazyslip.com,NaN
8984063,124604448,248,10894,2018-10-31 23:59:59,4,wish.com,NaN


In [4]:
# split data into 30 min buckets
BUCKET_SIZE = 30 * 60

start_time = data_filtered['used_at'].min()
buckets = data_filtered["used_at"].sub(start_time)
data_filtered['bucket'] = (buckets.dt.total_seconds() // BUCKET_SIZE).astype(int)
print("Rows:", len(data_filtered))
data_filtered.sort_values("used_at").iloc[np.r_[0:5, -5:0]]

Rows: 9127431


,web_visits_id,panelist_id,url,used_at,active_seconds,domain,subdomain,bucket
1369019,114016772,1849,498963,2018-10-01 00:00:00,8,nicequest.com,NaN,0
1315961,113973380,750,481401,2018-10-01 00:00:00,2,vasilisaroom.site,NaN,0
1232915,113894036,957,454324,2018-10-01 00:00:00,6,google.com,NaN,0
1520755,114158144,1982,552480,2018-10-01 00:00:00,2,facebook.com,NaN,0
1346500,113996676,1170,491134,2018-10-01 00:00:01,5,ebay-kleinanzeigen.de,NaN,0
8884856,124428860,1627,2918269,2018-10-31 23:59:58,16,ebay.de,NaN,1487
8944905,124539076,1067,241123,2018-10-31 23:59:58,10,gfk.de,ecpo.,1487
9091905,124804680,1226,813332,2018-10-31 23:59:58,452,mypersonalroutine.com,de.,1487
8938760,124531492,177,2934742,2018-10-31 23:59:59,6,crazyslip.com,NaN,1487
8984063,124604448,248,10894,2018-10-31 23:59:59,4,wish.com,NaN,1487


In [5]:
# Filter out buckets whose size is outside the 35th-85th percentiles range
LOWER_BOUND = 0.35
UPPER_BOUND = 0.85

bucket_sizes = data_filtered.groupby(["panelist_id", "bucket"]).size()
lower_bound = bucket_sizes.quantile(LOWER_BOUND)
upper_bound = bucket_sizes.quantile(UPPER_BOUND)
print(f"Lower bound: {lower_bound}, Upper bound: {upper_bound}")

data_good_buckets = data_filtered.groupby(["panelist_id", "bucket"]).filter(
    lambda x: lower_bound <= len(x) <= upper_bound
)
print("Rows:", len(data_good_buckets))
data_good_buckets.iloc[np.r_[0:5, -5:0]]

Lower bound: 8.0, Upper bound: 47.0
Rows: 4069727


,web_visits_id,panelist_id,url,used_at,active_seconds,domain,subdomain,bucket
1,111761051,1421,0,2018-10-05 21:41:22,10,ebesucher.de,NaN,235
2,111761051,1421,0,2018-10-05 21:45:36,5,ebesucher.de,NaN,235
3,111761051,1421,0,2018-10-05 21:42:22,181,ebesucher.de,NaN,235
4,111761055,1421,1,2018-10-05 21:47:57,2,klamm.de,NaN,235
5,111761055,1421,2,2018-10-05 21:47:40,1,klamm.de,NaN,235
9151238,124906196,1739,236,2018-10-31 16:52:12,105,facebook.com,NaN,1473
9151239,124906196,1739,236,2018-10-31 16:51:04,8,facebook.com,NaN,1473
9151240,124906200,1739,2997793,2018-10-31 16:50:10,2,goo.gl,NaN,1473
9151241,124906204,1739,2997794,2018-10-31 16:50:12,52,focus.de,NaN,1473
9151242,124906208,1739,2997795,2018-10-31 16:51:12,60,berliner-sonntagsblatt.de,NaN,1473


In [6]:
# Filter out buckets whose web page visits all occur within a 15 minute range
# We want traces to be at least somewhat spread out in time
REQUIRED_TIME_SPAN = 15 * 60

data_good_range = data_good_buckets.groupby(["panelist_id", "bucket"]).filter(
    lambda x: (x["used_at"].max() - x["used_at"].min()).total_seconds() >= REQUIRED_TIME_SPAN
)
print("Rows:", len(data_good_range))
data_good_range.iloc[np.r_[0:5, -5:0]]

Rows: 2673105


,web_visits_id,panelist_id,url,used_at,active_seconds,domain,subdomain,bucket
75,111762035,789,21,2018-10-05 16:53:08,4,amazon.de,NaN,225
82,111762043,789,27,2018-10-05 16:42:14,10,icloud.com,NaN,225
83,111762043,789,28,2018-10-05 16:31:04,2,icloud.com,NaN,225
84,111762043,789,29,2018-10-05 16:53:18,20,icloud.com,NaN,225
85,111762043,789,30,2018-10-05 16:52:49,5,icloud.com,NaN,225
9151238,124906196,1739,236,2018-10-31 16:52:12,105,facebook.com,NaN,1473
9151239,124906196,1739,236,2018-10-31 16:51:04,8,facebook.com,NaN,1473
9151240,124906200,1739,2997793,2018-10-31 16:50:10,2,goo.gl,NaN,1473
9151241,124906204,1739,2997794,2018-10-31 16:50:12,52,focus.de,NaN,1473
9151242,124906208,1739,2997795,2018-10-31 16:51:12,60,berliner-sonntagsblatt.de,NaN,1473


In [ ]:
# Keep up to MAX_TRACES_PER_USER buckets per panelist


def filter_buckets(df):
    grouped = df.groupby("bucket")
    if len(grouped) <= MAX_TRACES_PER_USER:
        return df
    else:
        indices = np.arange(grouped.ngroups)
        np.random.shuffle(indices)
        selected_buckets = indices[:MAX_TRACES_PER_USER]
        return df[grouped.ngroup().isin(selected_buckets)]


data_pruned = (
    data_good_range.groupby("panelist_id")
    .apply(filter_buckets, include_groups=False)
    .reset_index()
    .drop(columns=["level_1"])
)
print("Rows:", len(data_pruned))
data_pruned.iloc[np.r_[0:5, -5:0]]

Rows after sampling: 215313


,panelist_id,web_visits_id,url,used_at,active_seconds,domain,subdomain,bucket
0,0,111765915,781,2018-10-05 10:25:10,117,youtube.com,NaN,212
1,0,111765915,775,2018-10-05 10:09:21,12,youtube.com,NaN,212
2,0,111765915,781,2018-10-05 10:27:07,87,youtube.com,NaN,212
3,0,111765915,795,2018-10-05 10:09:17,4,youtube.com,NaN,212
4,0,111765915,214,2018-10-05 10:24:28,42,youtube.com,NaN,212
215308,2147,122893836,113790,2018-10-26 14:49:25,216,sfgame.de,s2.,1229
215309,2147,122893836,113791,2018-10-26 14:39:15,32,sfgame.de,s6.,1229
215310,2147,122893836,113791,2018-10-26 14:53:01,22,sfgame.de,s6.,1229
215311,2147,122893836,113792,2018-10-26 14:54:09,100,sfgame.de,s4.,1229
215312,2147,122893836,113790,2018-10-26 14:55:49,280,sfgame.de,s2.,1229


In [8]:
data_transformed = data_pruned.copy()
data_transformed["subdomain"] = data_transformed["subdomain"].fillna("")
data_transformed["url"] = data_transformed["subdomain"] + data_transformed["domain"]
data_transformed.drop(columns=["subdomain", "domain"], inplace=True)
data_transformed.iloc[np.r_[0:5, -5:0]]

,panelist_id,web_visits_id,url,used_at,active_seconds,bucket
0,0,111765915,youtube.com,2018-10-05 10:25:10,117,212
1,0,111765915,youtube.com,2018-10-05 10:09:21,12,212
2,0,111765915,youtube.com,2018-10-05 10:27:07,87,212
3,0,111765915,youtube.com,2018-10-05 10:09:17,4,212
4,0,111765915,youtube.com,2018-10-05 10:24:28,42,212
215308,2147,122893836,s2.sfgame.de,2018-10-26 14:49:25,216,1229
215309,2147,122893836,s6.sfgame.de,2018-10-26 14:39:15,32,1229
215310,2147,122893836,s6.sfgame.de,2018-10-26 14:53:01,22,1229
215311,2147,122893836,s4.sfgame.de,2018-10-26 14:54:09,100,1229
215312,2147,122893836,s2.sfgame.de,2018-10-26 14:55:49,280,1229


In [42]:
remap = {}
rate_limit = asyncio.Semaphore(5)


def simplify_domain(domain):
    # some domains include a subdomain that don't response to GET requests
    # we should retry them with their main domain
    parts = domain.split(".")
    if len(parts) > 2:
        return ".".join(parts[-2:])
    return None


async def build_mapping(domain, session):
    if domain not in remap:
        # prevent concurrent calls from both building the same mapping
        event = asyncio.Event()
        remap[domain] = event

        try:
            async with rate_limit:
                print(f"Fetching mapping for {domain}")
                headers = {
                    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:140.0) Gecko/20100101 Firefox/140.0",
                }
                if "facebook" in domain:
                    # Facebook returns 400 error when we fake our User-Agent
                    headers = None
                response = await session.get(f"http://{domain}", headers=headers)
            remap[domain] = response.url
            print(f"Mapping {domain} to {response.url}")
        except Exception as e:
            if str(e):
                print(f"Error fetching {domain}: {e}")
            else:
                print(f"Error fetching {domain}: {repr(e)}")

            simplified = simplify_domain(domain)
            if not simplified:
                print(f"Can't simplify {domain}, setting to None")
                remap[domain] = None
            else:
                await build_mapping(simplified, session)
                remap[domain] = remap[simplified]
                print(f"Simplified mapping {domain} to {remap[domain]}")

        event.set()
    elif isinstance(remap[domain], asyncio.Event):
        # caller expects this function to finish only after the mapping is built
        await remap[domain].wait()


async def build_all_mappings(unique_domains):
    async with aiohttp.ClientSession(
        connector=aiohttp.TCPConnector(
            ssl=False,
            force_close=True,
            limit=500,
        ),
        raise_for_status=True,
        auto_decompress=False,
        timeout=aiohttp.ClientTimeout(total=None, sock_connect=30, sock_read=30),
    ) as session:
        await asyncio.gather(*(build_mapping(domain, session) for domain in unique_domains))
    return remap

In [43]:
data_remapped = data_transformed.copy()
unique_domains = data_transformed["url"].unique()
mapping =  await build_all_mappings(unique_domains)
data_remapped["url"] = data_remapped["url"].map(mapping)

Fetching mapping for youtube.com
Fetching mapping for google.de
Fetching mapping for photos.google.com
Fetching mapping for tvline.com
Fetching mapping for emedien3.sub.uni-hamburg.de
Mapping tvline.com to https://tvline.com/
Fetching mapping for comicbook.com
Mapping comicbook.com to https://comicbook.com/
Fetching mapping for netflix.com
Mapping photos.google.com to https://www.google.com/photos/about/
Fetching mapping for nytimes.com
Mapping google.de to https://www.google.com/?gws_rd=ssl
Fetching mapping for tribpub.com
Mapping youtube.com to https://www.youtube.com/
Fetching mapping for theatlantic.com
Mapping emedien3.sub.uni-hamburg.de to http://emedien3.sub.uni-hamburg.de
Fetching mapping for nationalcouncilofchurches.us
Mapping theatlantic.com to https://www.theatlantic.com/
Fetching mapping for newsweek.com
Mapping nytimes.com to https://www.nytimes.com/
Fetching mapping for washingtonpost.com
Mapping tribpub.com to https://www.tribpub.com/
Fetching mapping for vice.com
Mappi

In [56]:
print("Rows:", len(data_pruned))
data_remapped.iloc[np.r_[0:5, -5:0]]

Rows: 215313


,panelist_id,web_visits_id,url,used_at,active_seconds,bucket
0,0,111765915,https://www.youtube.com/,2018-10-05 10:25:10,117,212
1,0,111765915,https://www.youtube.com/,2018-10-05 10:09:21,12,212
2,0,111765915,https://www.youtube.com/,2018-10-05 10:27:07,87,212
3,0,111765915,https://www.youtube.com/,2018-10-05 10:09:17,4,212
4,0,111765915,https://www.youtube.com/,2018-10-05 10:24:28,42,212
215308,2147,122893836,https://sfgame.net/,2018-10-26 14:49:25,216,1229
215309,2147,122893836,https://sfgame.net/,2018-10-26 14:39:15,32,1229
215310,2147,122893836,https://sfgame.net/,2018-10-26 14:53:01,22,1229
215311,2147,122893836,https://sfgame.net/,2018-10-26 14:54:09,100,1229
215312,2147,122893836,https://sfgame.net/,2018-10-26 14:55:49,280,1229


In [69]:
# Filter out all traces with website visits to invalid URLs
data_good_urls = data_remapped.groupby(["panelist_id", "bucket"]).filter(
    lambda x: x["url"].notna().all()
)
print(f"Rows: {len(data_good_urls)}")
print(f"Number of possible traces: {len(data_good_urls.groupby(['panelist_id', 'bucket']))}")
data_good_urls.iloc[np.r_[0:5, -5:0]]

Rows: 146354
Number of possible traces: 6259


,panelist_id,web_visits_id,url,used_at,active_seconds,bucket
0,0,111765915,https://www.youtube.com/,2018-10-05 10:25:10,117,212
1,0,111765915,https://www.youtube.com/,2018-10-05 10:09:21,12,212
2,0,111765915,https://www.youtube.com/,2018-10-05 10:27:07,87,212
3,0,111765915,https://www.youtube.com/,2018-10-05 10:09:17,4,212
4,0,111765915,https://www.youtube.com/,2018-10-05 10:24:28,42,212
215308,2147,122893836,https://sfgame.net/,2018-10-26 14:49:25,216,1229
215309,2147,122893836,https://sfgame.net/,2018-10-26 14:39:15,32,1229
215310,2147,122893836,https://sfgame.net/,2018-10-26 14:53:01,22,1229
215311,2147,122893836,https://sfgame.net/,2018-10-26 14:54:09,100,1229
215312,2147,122893836,https://sfgame.net/,2018-10-26 14:55:49,280,1229


In [74]:
data_good_urls["active_seconds"].describe()

count    146354.000000
mean         43.300504
std          88.062659
min           1.000000
25%           4.000000
50%          12.000000
75%          40.000000
max        3479.000000
Name: active_seconds, dtype: float64

In [73]:
data_good_urls.to_csv(f"{OUTPUT_FOLDER}/cleaned_buckets.csv", index=False)